In [1]:
import os

In [2]:
%pwd

'/Users/inamgurung/Desktop/Projects/AWS/Kidney-Disease-Classification/Kidney-Disease-Classification-Deep-Learning-Project/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/inamgurung/Desktop/Projects/AWS/Kidney-Disease-Classification/Kidney-Disease-Classification-Deep-Learning-Project'

In [5]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/Saphine/Kidney-Disease-Classification-Deep-Learning-Project.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="Saphine"
os.environ["MLFLOW_TRACKING_PASSWORD"]="cc2d6ec69549b739e9c9b02a95f344d41ded5461"


In [6]:
import tensorflow as tf

2026-01-20 15:05:22.555753: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [9]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [10]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath: Path = CONFIG_FILE_PATH,
        params_filepath: Path = PARAMS_FILE_PATH,
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model = "artifacts/training/model.h5",
            training_data = "artifacts/data_ingestion/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone",
            mlflow_uri = "https://dagshub.com/Saphine/Kidney-Disease-Classification-Deep-Learning-Project.mlflow",
            all_params = self.params,
            params_image_size = self.params.IMAGE_SIZE,
            params_batch_size = self.params.BATCH_SIZE
        )        
        return eval_config

In [11]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [12]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _valid_generator(self):
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )



    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":
                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [13]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_evaluation_config()
    model_evaluation = Evaluation(config=model_evaluation_config)
    model_evaluation.evaluation()
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2026-01-20 15:05:32,177 - INFO - yaml file: config/config.yaml loaded successfully]
[2026-01-20 15:05:32,185 - INFO - yaml file: params.yaml loaded successfully]
[2026-01-20 15:05:32,202 - INFO - Created directory at: artifacts]
Found 120 images belonging to 4 classes.


2026-01-20 15:05:33.257958: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


8/8 [==============================] - 18s 2s/step - loss: 40.6027 - accuracy: 0.2500
[2026-01-20 15:05:50,939 - INFO - JSON file saved at: scores.json]


2026/01/20 15:05:52 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2026-01-20 15:05:53,533 - WARNING - Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: /var/folders/kz/kl5h7crd0r12br7n4ny5yvv80000gn/T/tmpil9ke_00/model/data/model/assets
[2026-01-20 15:05:54,153 - INFO - Assets written to: /var/folders/kz/kl5h7crd0r12br7n4ny5yvv80000gn/T/tmpil9ke_00/model/data/model/assets]


/Users/inamgurung/miniconda/envs/kidney2/lib/python3.11/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
Registered model 'VGG16Model' already exists. Creating a new version of this model...
2026/01/20 15:06:46 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 6
Created version '6' of model 'VGG16Model'.
